# Objective:

This project entails building a Book Recommender System for users based on user-based and item-based collaborative filtering 
approaches

# About Book Crossing Dataset:

This dataset has been compiled by Cai-Nicolas Ziegler in 2004, and it comprises of three tables for users, books and ratings. 
Explicit ratings are expressed on a scale from 1-10 (higher values denoting higher appreciation) and implicit rating is expressed
by 0

# Dataset:
    
http://www2.informatik.uni-freiburg.de/~cziegler/BX/

# 1. Load Libraries and Data

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

books = pd.read_csv('books.csv',sep=';',error_bad_lines=False,encoding='latin-1')
users = pd.read_csv('users.csv', error_bad_lines=False,sep=';',encoding='latin-1')
ratings = pd.read_csv('ratings (1).csv',sep=';',error_bad_lines=False, encoding='latin-1')

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


In [2]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [3]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [4]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


# 2. Check no.of records and features given in each dataset, clean the data

In [5]:
# user
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
User-ID     278858 non-null int64
Location    278858 non-null object
Age         168096 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [6]:
users.shape

(278858, 3)

In [7]:
# ratings
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
User-ID        1149780 non-null int64
ISBN           1149780 non-null object
Book-Rating    1149780 non-null int64
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [8]:
ratings.shape

(1149780, 3)

In [9]:
# books
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
ISBN                   271360 non-null object
Book-Title             271360 non-null object
Book-Author            271359 non-null object
Year-Of-Publication    271360 non-null object
Publisher              271358 non-null object
Image-URL-S            271360 non-null object
Image-URL-M            271360 non-null object
Image-URL-L            271357 non-null object
dtypes: object(8)
memory usage: 16.6+ MB


In [10]:
books.shape

(271360, 8)

# 3. Check unique values of yearOfPublication

In [11]:
books['Year-Of-Publication'].unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

# 4. Check the rows having 'DK Publishing Inc' as yearOfPublication

In [12]:
books[books['Year-Of-Publication'] == 'DK Publishing Inc']

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,NaN
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,NaN


# 5. Change the datatype of yearOfPublication to 'int' and Drop NaNs in 'publisher' column

In [13]:
books = books[books['Year-Of-Publication'] != 'Gallimard']
for index,row in books.iterrows():
    if(row['Year-Of-Publication'] == 'DK Publishing Inc'):
        author = row['Book-Title'].split(':')[0]
        title = ''.join(row['Book-Title'].split(':')[1])

        row['Image-URL-L'] = row['Image-URL-M']
        row['Image-URL-M'] = row['Image-URL-S']
        row['Image-URL-S'] = row['Publisher']
        row['Publisher'] = row['Year-Of-Publication']
        row['Year-Of-Publication'] = row['Book-Author']
        row['Book-Title'] = title
        row['Book-Author'] = author

In [14]:
books['Year-Of-Publication'] = books['Year-Of-Publication'].astype('int64')

In [15]:
books.dropna(subset=['Publisher'], inplace=True)

# 6. Explore Users dataset

a. Get all unique values in ascending order for column Age

b. Values below 5 and above 90 do not make much sense for our book rating case...hence replace these by NaNs

c. Replace null values in column Age with mean

d. Change the datatype of Age to int

In [16]:
users['Age'].sort_values().unique()

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 113., 114., 115., 116., 118., 119., 123., 124., 127.,
       128., 132., 133., 136., 137., 138., 140., 141., 143., 146., 147.,
       148., 151., 152., 156., 157., 159., 162., 168., 172., 175., 183.,
       186., 189., 199., 200., 201., 204., 207., 20

In [17]:
users[(users['Age'] > 90) | (users['Age'] < 5)] = np.nan


In [18]:
users['Age'] = users['Age'].fillna(value = users['Age'].mean())

In [19]:
users['Age'].isna().sum()

0

# 7. Explore Ratings dataset

a. Check the shape

b. Ratings dataset should have books only which exist in our books dataset. Drop the remaining rows

c. Ratings dataset should have ratings from users which exist in users dataset. Drop the remaining rows

d. Consider only ratings from 1-10 and leave 0s in column bookRating

e. Find out which rating has been given highest number of times

In [20]:
ratings.shape

(1149780, 3)

In [21]:
ratings_new = ratings[ratings.ISBN.isin(books.ISBN)]
ratings_new.shape

(1031133, 3)

In [22]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [23]:
ratings_new = ratings_new[(ratings_new['Book-Rating'] > 0) & (ratings_new['Book-Rating'] <= 10)]
ratings_new['Book-Rating'].unique()

array([ 5,  3,  6,  7,  9,  8, 10,  1,  4,  2])

In [24]:
ratings_new['Book-Rating'].value_counts()

8     91804
10    71225
7     66402
9     60776
5     45355
6     31687
4      7617
3      5118
2      2375
1      1481
Name: Book-Rating, dtype: int64

In [25]:
ratings_new.shape

(383840, 3)

# 8. Collaborative Filtering Based Recommendation Systems

# a. For more accurate results only consider users who have rated atleast 100 books

In [26]:
value_counts = ratings_new['User-ID'].value_counts()
ratings_matrix = ratings_new[ratings_new['User-ID'].isin(value_counts[value_counts >= 100].index)]
ratings_matrix.shape

(103269, 3)

# b. Generate matrix table from explicit ratings table

In [27]:
pivot = ratings_matrix.pivot(index='User-ID', columns='ISBN', values='Book-Rating').fillna(0)
pivot.head()

ISBN,0000913154,0001046438,000104687X,0001047213,0001047973,000104799X,0001048082,0001053736,0001053744,0001055607,...,B000092Q0A,B00009EF82,B00009NDAN,B0000DYXID,B0000T6KHI,B0000VZEJQ,B0000X8HIE,B00013AX9E,B0001I1KOG,B000234N3A
User-ID,,,,,,,,,,,,,,,,,,,,,
2033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
userId = pivot.index
isbn = pivot.columns
print(userId)
print(isbn)

Int64Index([  2033,   2110,   2276,   4017,   4385,   5582,   6242,   6251,
              6543,   6575,
            ...
            269566, 270713, 271448, 271705, 273113, 274061, 274301, 275970,
            277427, 278418],
           dtype='int64', name='User-ID', length=449)
Index(['0000913154', '0001046438', '000104687X', '0001047213', '0001047973',
       '000104799X', '0001048082', '0001053736', '0001053744', '0001055607',
       ...
       'B000092Q0A', 'B00009EF82', 'B00009NDAN', 'B0000DYXID', 'B0000T6KHI',
       'B0000VZEJQ', 'B0000X8HIE', 'B00013AX9E', 'B0001I1KOG', 'B000234N3A'],
      dtype='object', name='ISBN', length=66572)


# c. Generate the predicted ratings using SVD with no.of singular values to be 50

In [36]:
from scipy.sparse.linalg import svds
pivot.shape

(449, 66572)

In [31]:
U,sigma, vT = svds(pivot,k=50)

In [32]:
sigma =np.diag(sigma)

In [34]:
user_predicted_ratings = U.dot(sigma.dot(vT))

In [40]:
df = pd.DataFrame(user_predicted_ratings, columns=pivot.columns, index=pivot.index)
df.head()

ISBN,0000913154,0001046438,000104687X,0001047213,0001047973,000104799X,0001048082,0001053736,0001053744,0001055607,...,B000092Q0A,B00009EF82,B00009NDAN,B0000DYXID,B0000T6KHI,B0000VZEJQ,B0000X8HIE,B00013AX9E,B0001I1KOG,B000234N3A
User-ID,,,,,,,,,,,,,,,,,,,,,
2033,0.025341,-0.002146,-0.001431,-0.002146,-0.002146,0.002971,-0.003920,0.007035,0.007035,0.012316,...,0.000180,0.000226,0.042081,-0.016804,-0.080028,0.004746,0.028314,0.000120,-0.001693,0.067503
2110,-0.010012,-0.003669,-0.002446,-0.003669,-0.003669,0.001075,0.001440,-0.003500,-0.003500,0.001612,...,-0.000363,0.000403,0.008142,0.001104,-0.029224,0.000999,0.002363,-0.000242,0.000029,-0.013059
2276,-0.015054,-0.015457,-0.010304,-0.015457,-0.015457,0.007281,-0.014033,0.011941,0.011941,0.011796,...,-0.000455,0.001907,0.047982,0.005737,0.117859,0.006945,0.003119,-0.000304,0.009009,-0.057692
4017,-0.021499,0.035602,0.023735,0.035602,0.035602,0.030307,0.024215,-0.001053,-0.001053,0.067579,...,0.002971,0.009912,0.086248,-0.008818,0.016154,0.028848,-0.000125,0.001981,0.031201,-0.046664
4385,0.002077,-0.007965,-0.005310,-0.007965,-0.007965,0.002947,0.003057,0.000231,0.000231,0.006080,...,0.002120,0.001597,-0.012181,0.009420,0.673459,0.002591,-0.008229,0.001413,0.004918,0.047773


# 9. Take a particular user_id

a. Lets find the recommendations for user with id 2110

b. Get the predicted ratings for userID 2110 and sort them in descending order

c. Create a dataframe with name user_data containing userID 2110 explicitly interacted books

d. Combine the user_data and and corresponding book data(book_data) in a single dataframe with name user_full_info

In [45]:
reccomendation = df.loc[2110].sort_values(ascending=False)

In [47]:
reccomendation = reccomendation.sort_values(ascending=False)

In [51]:
user_data = ratings_matrix[ratings_matrix['User-ID'] == 2110]
user_data.shape

(103, 3)

In [57]:
user_full_info=pd.merge(user_data, books,how='inner', on='ISBN')
user_full_info.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,2110,0060987529,7,Confessions of an Ugly Stepsister : A Novel,Gregory Maguire,2000,Regan Books,http://images.amazon.com/images/P/0060987529.0...,http://images.amazon.com/images/P/0060987529.0...,http://images.amazon.com/images/P/0060987529.0...
1,2110,0064472779,8,All-American Girl,Meg Cabot,2003,HarperTrophy,http://images.amazon.com/images/P/0064472779.0...,http://images.amazon.com/images/P/0064472779.0...,http://images.amazon.com/images/P/0064472779.0...
2,2110,0140022651,10,Journey to the Center of the Earth,Jules Verne,1965,Penguin Books,http://images.amazon.com/images/P/0140022651.0...,http://images.amazon.com/images/P/0140022651.0...,http://images.amazon.com/images/P/0140022651.0...
3,2110,0142302163,8,The Ghost Sitter,Peni R. Griffin,2002,Puffin Books,http://images.amazon.com/images/P/0142302163.0...,http://images.amazon.com/images/P/0142302163.0...,http://images.amazon.com/images/P/0142302163.0...
4,2110,0151008116,5,Life of Pi,Yann Martel,2002,Harcourt,http://images.amazon.com/images/P/0151008116.0...,http://images.amazon.com/images/P/0151008116.0...,http://images.amazon.com/images/P/0151008116.0...


# 10. Get top 10 recommendation for above given userID

In [77]:
user_id = 2110
recommended_books = books[~books['ISBN'].isin(user_full_info['ISBN'])].merge(pd.DataFrame(reccomendation).reset_index(), 
                                                         how='left', left_on='ISBN', right_on='ISBN').rename(columns = {user_id: 'Predictions'})

In [78]:
recommended_books.shape

(271254, 9)

In [80]:
recommended_books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Predictions
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,NaN
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,0.000883
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,NaN
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,-0.001017
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,NaN


In [82]:
recommended_books.sort_values(by='Predictions', ascending=False).head(10)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Predictions
1192,0345370775,Jurassic Park,Michael Crichton,1999,Ballantine Books,http://images.amazon.com/images/P/0345370775.0...,http://images.amazon.com/images/P/0345370775.0...,http://images.amazon.com/images/P/0345370775.0...,0.368946
6184,0345384911,Crystal Line,Anne McCaffrey,1993,Del Rey Books,http://images.amazon.com/images/P/0345384911.0...,http://images.amazon.com/images/P/0345384911.0...,http://images.amazon.com/images/P/0345384911.0...,0.333624
5458,043935806X,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...,0.333209
455,044021145X,The Firm,John Grisham,1992,Bantam Dell Publishing Group,http://images.amazon.com/images/P/044021145X.0...,http://images.amazon.com/images/P/044021145X.0...,http://images.amazon.com/images/P/044021145X.0...,0.329336
2031,0451151259,Eyes of the Dragon,Stephen King,1988,Penguin Putnam~mass,http://images.amazon.com/images/P/0451151259.0...,http://images.amazon.com/images/P/0451151259.0...,http://images.amazon.com/images/P/0451151259.0...,0.313295
5383,0439139597,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2000,Scholastic,http://images.amazon.com/images/P/0439139597.0...,http://images.amazon.com/images/P/0439139597.0...,http://images.amazon.com/images/P/0439139597.0...,0.305088
3413,0439064872,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,2000,Scholastic,http://images.amazon.com/images/P/0439064872.0...,http://images.amazon.com/images/P/0439064872.0...,http://images.amazon.com/images/P/0439064872.0...,0.290587
976,0380759497,Xanth 15: The Color of Her Panties,Piers Anthony,1992,Eos,http://images.amazon.com/images/P/0380759497.0...,http://images.amazon.com/images/P/0380759497.0...,http://images.amazon.com/images/P/0380759497.0...,0.278563
2435,0345353145,Sphere,MICHAEL CRICHTON,1988,Ballantine Books,http://images.amazon.com/images/P/0345353145.0...,http://images.amazon.com/images/P/0345353145.0...,http://images.amazon.com/images/P/0345353145.0...,0.250941
6048,0451167317,The Dark Half,Stephen King,1994,Signet Book,http://images.amazon.com/images/P/0451167317.0...,http://images.amazon.com/images/P/0451167317.0...,http://images.amazon.com/images/P/0451167317.0...,0.249254
